# Task. Benchmark 任务

* 1. 预测是否是酶
* 2. 预测是几功能酶
* 3. 分别是什么功能

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-07-22  

## 任务简介
该任务通过给定蛋白序列，预测该该蛋白是酶还是非酶，是酶的话有几个功能，这些功能分别是什么。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后完成对应任务。


## 数据统计
- 训练数据取之2018年2月份的快照数据，该数据包括序列566,825条，安装核苷酸序列去除重复后还剩余469,129条唯一的数据。在这些数据中，其中没有酶号的数据为246,5625条，有酶号的数据为222,567条。训练数据涉及4855个独立的EC编号，其中4734个具有4级完整的EC号， 164个三级EC号，37个二级EC号，以及6个一级EC号。
- 训练集涉及的多功能能酶与不同EC级别对应的序列数量如下

| 酶功能数 | 序列条数   | EC精确级别 | 序列条数   |
|-----:|-------:|-------:|-------:|
| 1    | 209729 | 1      | 191128 |
| 2    | 10762  | 2      | 23720  |
| 3    | 934    | 3      | 6097   |
| 4    | 652    | 4      | 1622   |
| 5    | 199    |        |        |
| 6    | 101    |        |        |
| 7    | 124    |        |        |
| 8    | 18     |        |        |
| 9    | 14     |        |        |
| 10   | 25     |        |        |
|      | 222558 |        | 222567 （因为多功能酶有多个EC编号，故序列条数大于左边）|



- 测试集采用2020年6月份快照数据，构建方法为  <font color=red size=4>筛选</font>  在2020年6月份快照数据中出现的序列 <font color=red size=6>且</font> 未在2018年2月份快照数据中出现的序列。最终序列条数为8033条，设计1036个独立的EC编号， 其中有728个EC编号在训练数据中出现，308个新增EC号，这些新增EC号EC迁移的有188个，纯新增的有120个。

 


## 1. 导入必要的包

In [2]:
import numpy as np
import pandas as pd
import random
from random import randint
import time
import gzip
import re
import datetime
import sys
import os
from tqdm import tqdm

from functools import reduce
import matplotlib.pyplot as plt
import joblib


sys.path.append("../../tools/")
import commontools
import funclib
from pyecharts.globals import CurrentConfig, OnlineHostType
CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType

%load_ext autoreload
%autoreload 2

## 2. 定义公共数据

In [15]:
#定义数据目录
DATADIR = r'''/home/shizhenkun/codebase/BioUniprot/data/benchmark/data/'''
RESULTSDIR = r'''/home/shizhenkun/codebase/BioUniprot/data/benchmark/results/'''
MODELDIR = r'''/home/shizhenkun/codebase/BioUniprot/data/benchmark/model/'''


#导入标签字典
dict_ec_label = np.load(DATADIR+'ec_label_dict.npy', allow_pickle=True).item()
dict_label_ec = dict(zip(dict_ec_label.values(), dict_ec_label.keys()))

# 读入数据
train = pd.read_feather(DATADIR+'train.feather')
test = pd.read_feather(DATADIR+'test.feather')


train_fasta = DATADIR+'train.fasta'
test_fasta = DATADIR+'test.fasta'

In [28]:
def get_blast_pred(train, test, blast_res):
    # 给比对结果添加标签
    isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
    blast_res['isEmzyme_pred'] = blast_res['sseqid'].apply(lambda x: isEmzyme_dict.get(x))
    
    isEmzyme_dict_test = {v: k for k,v in zip(test.isemzyme, test.id )} 
    blast_res['isEmzyme_groudtruth'] = blast_res['id'].apply(lambda x: isEmzyme_dict_test.get(x))
    return blast_res

## 3. 酶非酶预测

In [29]:
blast_res = funclib.getblast_fasta(train_fasta, test_fasta) #序列比对
blast_res = get_blast_pred(train, test, blast_res)          #添加序列比对标签

diamond makedb --in /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [30]:
blast_res

,id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,isEmzyme_pred,isEmzyme_groudtruth
0,P02462,P02462,99.8,1669,4,0,1,1669,1,1669,6.370000e-149,503.0,False,False
1,P00939,P00939,100.0,248,0,0,2,249,1,248,4.870000e-182,501.0,True,True
2,P02340,P02340,100.0,387,0,0,4,390,1,387,5.320000e-286,776.0,False,False
3,P01848,P01848,100.0,140,0,0,1,140,3,142,4.280000e-96,275.0,False,False
4,P01733,P01733,100.0,114,0,0,1,114,1,114,1.840000e-82,239.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6030,Q6YZW2,Q9LQI9,64.5,403,120,9,51,446,32,418,5.480000e-168,481.0,False,False
6031,A0A061ACU2,M9MSG8,26.1,2702,1523,77,13,2435,12,2518,2.870000e-196,665.0,False,False
6032,Q6NRV0,Q9BWF2,56.4,470,195,5,1,463,1,467,1.490000e-144,424.0,True,True
6033,C5DLH0,Q6CNF8,63.1,385,137,3,1,381,1,384,3.490000e-180,508.0,True,True


In [57]:
# 获取比对结果
res_data_sub = blast_res[['id','isEmzyme_pred']]

#选取测试集的ID与是否是酶2列
test_data_sub=test[['id', 'isemzyme']]


#拼合比对结果到测试集
final_pred_table = pd.merge(test_data_sub,res_data_sub, on='id', how='left')
final_pred_table = final_pred_table.rename(columns={'isEmzyme_pred':'isEmzyme_pred_blast'})

In [58]:
def caculate_isEnzymeBlastMetrics(pred_table):
    
    res = pred_table
    
    # 填充没有比对结果的值为非酶
    res.fillna(False, inplace=True)

    # # # 填充没有比对结果的值为酶
#     res.fillna(True, inplace=True)

    # 填充没有比对结果的值 随机填充
#     list_enornot=[True, False]
#     for i in range(len(res[(res.isEmzyme_pred_blast.isnull())])):
#         res.fillna(random.choice(list_enornot), inplace=True, limit=1)

    # 计算指标
    tp = len(res[res.isemzyme & res.isEmzyme_pred_blast])
    fp = len(res[(res.isemzyme ==False) & (res.isEmzyme_pred_blast)])
    tn = len(res[(res.isemzyme ==False) & (res.isEmzyme_pred_blast ==False)])
    fn = len(res[(res.isemzyme ) & (res.isEmzyme_pred_blast == False)])
    print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
    funclib.caculateMetrix_1('同源比对',tp, fp, tn,fn)

In [59]:
caculate_isEnzymeBlastMetrics(final_pred_table)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
同源比对 	 0.902776  	0.969148 		 0.863591  	0.807488	 0.880963		 	 	 	 tp:2890  fp:92  fn:689  tn:4362


In [67]:
X_train = train.iloc[:,4:]
X_test = test.iloc[:,4:]
Y_train = train.iloc[:,1].astype('int')
Y_test = test.iloc[:,1].astype('int')
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.837296 	0.860406 		0.822372 	0.757754 	0.805824 	0.915852 	0.897813 	 tp: 2712 fp: 440 fn: 867 tn: 4014
xg 		0.870908 	0.901199 		0.851182 	0.797709 	0.846302 	0.941582 	0.929918 	 tp: 2855 fp: 313 fn: 724 tn: 4141
dt 		0.794846 	0.792842 		0.796242 	0.730372 	0.760326 	0.788513 	0.699199 	 tp: 2614 fp: 683 fn: 965 tn: 3771
rf 		0.863438 	0.913116 		0.833764 	0.766415 	0.833359 	0.941952 	0.928926 	 tp: 2743 fp: 261 fn: 836 tn: 4193
gbdt 		0.834184 	0.873877 		0.810461 	0.733725 	0.797691 	0.915952 	0.899464 	 tp: 2626 fp: 379 fn: 953 tn: 4075
